<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Neural Network</div></h1>

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import pickle

### dl & clean & preprocess data

##### dl

In [2]:
path_to_zip = keras.utils.get_file('spa-eng.zip',
                                   origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
                                   extract=True)
path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng', 'spa.txt')

##### clean

In [3]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)       # expand spase ali. --> ali .
    w = re.sub(r'[" "]+', " ", w)             # remove extra space
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)     # revove all - [^a-zA-Z?.~,]
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w
print(unicode_to_ascii("Aún no he dicho nada. !&^%"))
print(preprocess_senetence("Aún no he dicho nada. !&^%"))

Aun no he dicho nada. !&^%
<start> aun no he dicho nada . <end>


In [4]:
def create_dataset(path, num_examples):
    
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    
    return zip(*word_pairs)

print(list(create_dataset(path_to_file, 2)))

[('<start> go . <end>', '<start> go . <end>'), ('<start> ve . <end>', '<start> vete . <end>')]


##### preprocess

In [5]:
def max_length(tensor):
    return max(len(t) for t in tensor)

def tokenize(lang):
    # tokenize
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    
    # transfirm input
    tensor = lang_tokenizer.texts_to_sequences(lang)
    
    # pading --> max len
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

# call all fun
def load_dataset(path, num_examples=None):
    
    targ_lang, inp_lang = create_dataset(path, num_examples)
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

#decoder
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, None)
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
# print(target_lang_tokenizer.word_index)
print(input_tensor[0])
print(convert(input_lang_tokenizer, input_tensor[0]))

with open('../../__data/Translation Attention Networks/input_lang_tokenizer.moh','wb') as f:
    pickle.dump(input_lang_tokenizer,f)
with open('../../__data/Translation Attention Networks/target_lang_tokenizer.moh','wb') as f:
    pickle.dump(target_lang_tokenizer,f)

[ 1 40  3  2  0  0  0  0  0]
1  ....  <start>
40  ....  ve
3  ....  .
2  ....  <end>
None


#### create dataset

In [6]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

### create and fit model
##### create

In [7]:
example_input_batch, example_target_batch = next(iter(dataset))
max_length_targ, max_length_inp, len(input_lang_tokenizer.word_index), len(target_lang_tokenizer.word_index)

(7, 9, 752, 346)

In [8]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        # print(x.shape)
        x = self.embedding(x)
        # print(x.shape)
        output, state = self.gru(x, initial_state=hidden)
        # print(output.shape)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

simple_hidden = encoder.initilize_hidden_state()
print(simple_hidden[:2])

simple_output, simple_states = encoder(example_input_batch, simple_hidden)
simple_output.shape

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(2, 1024), dtype=float32)


TensorShape([64, 9, 1024])

In [9]:
# attention use decoder
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        # print(query.shape)  # last hiden state 
        # print(values.shape) # encoder
        hidden_with_time_axis = tf.expand_dims(query, 1)
        # print(hidden_with_time_axis.shape)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        # print((self.W1(values)).shape)
        # print((self.W2(hidden_with_time_axis)).shape)
        # print(score.shape)
        atteion_weights = tf.nn.softmax(score, axis=1)
        # print(atteion_weights.shape)
        context_vector = atteion_weights * values
        # print(context_vector.shape)
        context_vector = tf.reduce_sum(context_vector, axis=1)
        # print(context_vector.shape)
        return context_vector, atteion_weights
    
attention_layer = Attention(units)

attention_result, _ = attention_layer(simple_hidden, simple_output)

In [10]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        # print(x.shape)
        context_vector, _ = self.attention(hidden, enc_output)
        # print(context_vector.shape)
        x = self.embedding(x)
        # print(x.shape)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # print(x.shape)
        output, state = self.gru(x)
        # print(output.shape)
        output = tf.reshape(output, (-1, output.shape[2]))
        # print(output.shape)
        x = self.fc(output)
        #print(x.shape)
        return x, state
    
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

x, state = decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

##### fit

In [11]:
optimizer = keras.optimizers.Adam()

loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [12]:
checkpoint_dir = '../../__data/Translation Attention Networks/Translation_Attention'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [13]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [ ]:
EPOCH = 1
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
    loss = total_loss/steps_per_epoch
    print('Epoch: ', epoch)
    print('Loss: ', loss.numpy())
    checkpoint.save(file_prefix=checkpoint_dir)

In [14]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [47]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
with open('../../__data/Translation Attention Networks/input_lang_tokenizer.moh','rb') as f:
    input_lang_tokenizer = pickle.load(f)
with open('../../__data/Translation Attention Networks/target_lang_tokenizer.moh','rb') as f:
    target_lang_tokenizer = pickle.load(f)
evaluate('hace mucho frio aqui.')

('i m not a car . <end> ', '<strat> hace mucho frio aqui . <end>')

<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Neural Network</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>